In [1]:
from IPython.display import display, clear_output, HTML
import time 
import random 
import math
import requests
from bs4 import BeautifulSoup
import json
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

In [2]:
def send_to_google_form(data_dict, form_url):
    
    #send results to google form 
    
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [3]:
event_info = {
'type': '',
'description': '',
'time': -1
}

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    print(f"Button clicked: {btn.description}")
    
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
    
    start_wait = time.time()

    # with ui_events() as poll:
    #     while True:
    #         poll()  # Process UI events

    #         # Check for a button click event
    #         if event_info['description'] != "":
    #             break
    #         # Optionally, break after a timeout
    #         if timeout != -1 and time.time() > start_wait + timeout:
    #             break
            #time.sleep(interval)
    n_proc = int(max_rate*interval)+1

    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            
            time.sleep(interval)

    return event_info

#yes button function
def on_button_click():
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfn4FE78zFmlhYG1rXco5Kw3rBj9Xg0QGVHSPoItT0U5SyXVA/viewform?usp=sf_link"
    
    data_dict = {
        'name': user_id,
        'gender': gender, 
        'age': age,
        'score': score,
        'total_time': total_time
    }
    print("Thanks - your data will be uploaded.")
    send_to_google_form(data_dict, form_url)

#no button function
def register_event():
    print("No problem, we hope you enjoyed the test.")




In [4]:
def display_intro(): 

    #display the title and introduction to the test 
    
    title = "Mathematical Ability Test"
    print(title)
    
    time.sleep(1)
    
    print("_" * len(title))
    
    time.sleep(1)

    #global the data from this function for data collection 
    global user_id
    global gender 
    global age 
    
    id_instructions = """

    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifer would be CBTC
    
    """
    
    print(id_instructions)
    
    user_id = input("> ")
    
    print("User entered id:", user_id)
     
    time.sleep(2)
    clear_output()

    print("Please enter your gender:")
    gender = input(">")

    print("Please enter your age:")
    age = input(">")

    time.sleep(2)
    clear_output()

    welcome_message = (f"Welcome to the Mathematical Ability Test {user_id}! Do it as fast and as accurate as possible. GOOD LUCK!\nP.S. NO CALCULATOR!!!!")
    print(welcome_message)

    time.sleep(5)
    
    clear_output()
    
    return welcome_message, user_id

In [ ]:
def math_ability_test(): 
    
    global score
    global total_time

    score = 0 
    
    display_intro()

    start_time = time.time()
        
    time.sleep(1)

    #list of questions with increasing difficulty 
    questions_list = ["16 + 3", "3 + 5 + 18", "12 + 30 x 2"] 
                     #"27 + 4 + 12", "12 x 2 + 43", "30 - 25 + 18"] 
                     # "16 x 3 + 3", "25 + 8 x 9", "57 - 22 x 2", 
                     #  "92 - 40 x 3", "12 x 4 + 56", "83 x 3 - 22",
                     #  "52 ÷ 2 x 12", "282 - 15 x 3", "34 x 5 ÷ 2",
                     #  "21 + 5 x 12 x 2", "52 ÷ 4 x 3 + 213", 
                     #  "23 x 3 x 9", "520 x 3 - 9 + 12", "123 + 456 ÷ 24 x 2"]
    
    style = f"font-size: 50px;"
    question_num = 0 

    #run through each question in the question_list 
    for question in questions_list: 
        
        question_num +=1

        try:
            # Replace "x" with "*"
            replaced_ques = question.replace("x", "*")
            # Replace "÷" with "/"
            replaced_ques = replaced_ques.replace("÷", "/")

            ans = eval(replaced_ques)
            ans = int(ans)
        except SyntaxError:
            # Handle other exceptions if needed
            pass

        #print(ans)

        #visualise the questions in larger font 
        question_number = HTML(f"""<span style= 20px>Question number: {question_num}</span>""")
        visualise_expression = HTML(f"""<span style ='{style}'>{question}</span>""")
        
        display(question_number)
        display(visualise_expression) 

        user_input = input("What's your answer?")
    
        ans_input = int(user_input)

        time.sleep(1)
        clear_output()

        #check if answer is correct 
        if ans_input == ans:
            score = score + 1
            print("Well done!")
            time.sleep(1)
            clear_output()
        else: 
            print("Try harder next time!")
            time.sleep(1)
            clear_output()
            
        end_time = time.time()
        time_taken = end_time - start_time 
                    
        total_time = 0 
        total_time = total_time + time_taken 

    #judging matrix: speed and accuracy 
    test_result = f"Score: {score}, time taken: {total_time}"
    print(test_result) 

    #data collection consent with buttons
        
    print("Please read:")
    print("")
    print("we wish to record your response data")
    print("to an anonymised public data repository. ")
    print("Your data will be used for educational teaching purposes")
    print("practising data analysis and visualisation.")
    print("")
    print("Please click yes in the button below if you consent to the upload.")
    
    btn_yes = widgets.Button(description="Yes")
    btn_no = widgets.Button(description="No")
    
    btn_yes.on_click(register_btn_event)
    btn_no.on_click(register_btn_event)

    
    panel = widgets.HBox([btn_yes, btn_no])
    display(panel)
    #wait_for_event()

    event = wait_for_event()
    if event['description'] == "Yes":
        on_button_click()  # Function to handle "Yes" button click
    elif event['description'] == "No":
        register_event()  # Function to handle "No" button click

    print("That's the end of the test. Have a nice day!")
    clear_output(wait=False)
    
    return test_result
        
#call to run test 
math_ability_test()
random.seed(1)


What's your answer? 19
